# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
# Calculating the number of viewers and clickers
viewers = df[df['action'] == 'view']
clickers = df[df['action'] == 'click']

num_viewers = len(viewers)
num_clickers = len(clickers)

print(f"Total Viewers: {num_viewers}")
print(f"Total Clickers: {num_clickers}")


Total Viewers: 6328
Total Clickers: 1860


In [3]:
# Checking for overlap between control and experiment groups
control_group = df[df['group'] == 'control']
experiment_group = df[df['group'] == 'experiment']

overlap = len(set(control_group['id']).intersection(experiment_group['id']))

print(f"Number of Users in Both Control and Experiment Groups: {overlap}")


Number of Users in Both Control and Experiment Groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [4]:
'''Your code here
# Calculate CTR for control group
control_clicks = len(control_group[control_group['action'] == 'click'])
control_ctr = control_clicks / len(control_group)

# Calculate CTR for experiment group
experiment_clicks = len(experiment_group[experiment_group['action'] == 'click'])
experiment_ctr = experiment_clicks / len(experiment_group)

print(f"Control Group CTR: {control_ctr:.4f}")
print(f"Experiment Group CTR: {experiment_ctr:.4f}")'''


'Your code here\n# Calculate CTR for control group\ncontrol_clicks = len(control_group[control_group[\'action\'] == \'click\'])\ncontrol_ctr = control_clicks / len(control_group)\n\n# Calculate CTR for experiment group\nexperiment_clicks = len(experiment_group[experiment_group[\'action\'] == \'click\'])\nexperiment_ctr = experiment_clicks / len(experiment_group)\n\nprint(f"Control Group CTR: {control_ctr:.4f}")\nprint(f"Experiment Group CTR: {experiment_ctr:.4f}")'

In [5]:
# Count the number of click and non-click events in the control and experiment groups
control_clicks = len(control_group[control_group['action'] == 'click'])
control_non_clicks = len(control_group[control_group['action'] == 'view']) - control_clicks

experiment_clicks = len(experiment_group[experiment_group['action'] == 'click'])
experiment_non_clicks = len(experiment_group[experiment_group['action'] == 'view']) - experiment_clicks

# Create the contingency table as a nested list
contingency_table = [
    [control_clicks, control_non_clicks],
    [experiment_clicks, experiment_non_clicks]
]

# Print the contingency table
for row in contingency_table:
    print(row)


[932, 2400]
[928, 2068]


In [9]:
from scipy.stats import chi2_contingency

# Contingency table
contingency_table = [
    [control_clicks, control_non_clicks],
    [experiment_clicks, experiment_non_clicks]
]

# Perform the chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print(f"Chi-squared statistic: {chi2:.4f}")
print(f"P-value: {p:.4f}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies Table:")
for row in expected:
    print(row)


Chi-squared statistic: 6.7129
P-value: 0.0096
Degrees of Freedom: 1
Expected Frequencies Table:
[ 979.38053097 2352.61946903]
[ 880.61946903 2115.38053097]


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [10]:
#Your code here
# Calculate the pooled click-through rate
pooled_ctr = (control_clicks + experiment_clicks) / (len(control_group) + len(experiment_group))

# Calculate the expected number of clicks for the experiment group
expected_clicks_experiment = len(experiment_group) * pooled_ctr


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [11]:
#Your code here
import math

# Calculate the standard error using the binomial variance formula
n_experiment = len(experiment_group)
variance_experiment = n_experiment * pooled_ctr * (1 - pooled_ctr)
standard_error = math.sqrt(variance_experiment)

# Calculate the z-score
z_score = (experiment_clicks - expected_clicks_experiment) / standard_error


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [12]:
#Your code here
from scipy.stats import norm

# Calculate the p-value
p_value = 1 - norm.cdf(z_score)  # Use 1 - CDF to get the upper tail probability

print(f"Expected Clicks for Experiment Group: {expected_clicks_experiment:.2f}")
print(f"Z-Score: {z_score:.4f}")
print(f"P-Value: {p_value:.4f}")


Expected Clicks for Experiment Group: 891.38
Z-Score: 1.3951
P-Value: 0.0815


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

The chi-squared test results suggest a statistically significant difference, while the z-test results do not provide strong evidence for such a difference.
The p-values from the two tests are substantially different (0.0096 vs. 0.0815).

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.